# Berkeley Course Crawler

This crawler helps you extract more detailed information from Berkeley Course webpage. Follow the steps below, and have your excel/google sheet/database prepared. You can organize your short list of courses.

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jul  4 23:32:27 2020

@author: You, Bo-Xiang
"""
import requests
from datetime import datetime, timedelta
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
from IPython.display import display, HTML
import calendar

class course_info():
    def __init__(self, url_list):
        
        # if the input is not a list, convert it to list
        if type(url_list) is not list:
            url_list = [url_list]
        
        self.information = pd.DataFrame(columns=["Course_No.","Serial_NO.","Course_Name","Location",
                                                 "Start_Time(SF_time)","End_Time(SF_time)","Start_Time(TW_time)",
                                                 "End_Time(TW_time)","Level","Type","Mode","Instructor(s)",
                                                 "Units","Total_Capacity","Total_Enrolled",
                                                 "Final_Examination","Description"])
        for url in url_list:
            self.course_extract(url)
            time.sleep(0.8);
    
    # Method 1: 
    def course_extract(self, url):
        
            r = requests.get(url)
            soup = BeautifulSoup(r.text, "html.parser")
            query = soup.find("div", class_="handlebarData theme_is_whitehot").attrs
            js_str = query["data-json"]
            js_dict = json.loads(js_str)
            course_num = js_dict["displayName"]
            serial_num = js_dict["id"]
            name = js_dict["course"]["title"]
            location = js_dict["meetings"][0]["location"]["description"]
            level = js_dict["course"]["academicCareer"]["description"]
            lecture = js_dict["component"]["description"]
            capacity = js_dict["enrollmentStatus"]["maxEnroll"]
            enrolled = js_dict["enrollmentStatus"]["enrolledCount"]
            
            try:
                special_title = js_dict["attributes"]["NOTE"]["special-title"]["value"]["formalDescription"]
                name = name + " : " + special_title
            except KeyError:
                name = name
        
            try:
                mode = js_dict["attributes"]["WEB"][0]["value"]["formalDescription"]
            except KeyError:
                mode = "Pending Reviews"
                
            try:
                instructor = js_dict["meetings"][0]["assignedInstructors"]
                instructors = ', '.join([str(i["instructor"]["names"][1]["formattedName"]) for i in instructor])
            except KeyError:
                instructors = "None"
            
            try:
                units = js_dict["course"]["credit"]["value"]["fixed"]["units"]
            except KeyError:
                units = ' to '.join([str(js_dict["course"]["credit"]["value"]["range"]["minUnits"]), str(js_dict["course"]["credit"]["value"]["range"]["maxUnits"])])
                
            # date process
            fake_date = []
            if js_dict["meetings"][0]["meetsMonday"]:
                fake_date.append(datetime(2020,7,6).date())
            if js_dict["meetings"][0]["meetsTuesday"]:
                fake_date.append(datetime(2020,7,7).date())
            if js_dict["meetings"][0]["meetsWednesday"]:
                fake_date.append(datetime(2020,7,8).date())
            if js_dict["meetings"][0]["meetsThursday"]:
                fake_date.append(datetime(2020,7,9).date())
            if js_dict["meetings"][0]["meetsFriday"]:
                fake_date.append(datetime(2020,7,10).date())
            fake_starttime = datetime.strptime(js_dict["meetings"][0]["startTime"], '%H:%M:%S').time()
            fake_endtime = datetime.strptime(js_dict["meetings"][0]["endTime"], '%H:%M:%S').time()
            fake_start_dt = [datetime.combine(i, fake_starttime) for i in fake_date]
            fake_end_dt = [datetime.combine(i, fake_endtime) for i in fake_date]
            
            sf_start_dt = " / ".join([str(i.strftime("%a %H:%M")) for i in fake_start_dt])
            sf_end_dt = " / ".join([str(i.strftime("%a %H:%M")) for i in fake_end_dt])
            
            tw_start_dt = " / ".join([str((i + timedelta(hours=15)).strftime("%a %H:%M")) for i in fake_start_dt])
            tw_end_dt = " / ".join([str((i + timedelta(hours=15)).strftime("%a %H:%M")) for i in fake_end_dt])
            
            description = js_dict["course"]["description"]
            final = js_dict["course"]["finalExam"]["description"]
            
            self.information = self.information.append({
                    "Course_No." : course_num,
                    "Serial_NO." : serial_num,
                    "Course_Name" : name,
                    "Location" : location,
                    "Start_Time(SF_time)" : sf_start_dt,
                    "End_Time(SF_time)" : sf_end_dt,
                    "Start_Time(TW_time)" : tw_start_dt,
                    "End_Time(TW_time)" : tw_end_dt,
                    "Level" : level,
                    "Type" : lecture,
                    "Mode" : mode,
                    "Instructor(s)" : instructors,
                    "Units" : units,
                    "Total_Capacity" : capacity,
                    "Total_Enrolled" : enrolled,
                    "Final_Examination" : final,
                    "Description" : description}, ignore_index=True, sort=False)
    # method 2:
    def weekly_schedule(self, time_zone="SF"):
        info = self.information
        # define time zone
        if time_zone == "SF":
            start_col = "Start_Time(SF_time)"
            end_col = "End_Time(SF_time)"
        
        elif time_zone == "TW":
            start_col = "Start_Time(TW_time)"
            end_col = "End_Time(TW_time)"
        
        else:
            print('Error: time_zone must be "SF" or "TW".')
            return 

        schedule = pd.DataFrame({
                "Monday": "",
                "Tuesday": "",
                "Wednesday": "",
                "Thursday": "",
                "Friday": "",
                "Saturday": ""},
                index = pd.date_range("00:00", "23:30", freq="30min").time)
        
        for i in range(0, info.shape[0]):

            str_time = info.loc[i, start_col].split(" / ")
            end_time = info.loc[i, end_col].split(" / ")
            course_name = info.loc[i, "Course_Name"]
            for t in range(0, len(str_time)):
                str_formatted = datetime(*time.strptime(str_time[t], "%a %H:%M")[:7])
                end_formatted = datetime(*time.strptime(end_time[t], "%a %H:%M")[:7])
                DoW = calendar.day_name[time.strptime(str_time[t], "%a %H:%M")[6]]
                str_point = str_formatted.strftime("%H:%M")
                end_point = end_formatted.strftime("%H:%M")
                period = pd.date_range(str_point, end_point, freq="30min").time
                schedule.loc[period, DoW] = schedule.loc[period, DoW] + " \n " + course_name
        display(HTML(schedule.to_html().replace("\\n","<br>")))
        return schedule

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

## Demonstration

### 1. Export information of course of your interest

In [3]:
# Example for three selected Course: 
url_list = ["https://classes.berkeley.edu/content/2020-spring-geog-149b-001-lec-001",
       "https://classes.berkeley.edu/content/2020-spring-geog-167ac-001-lec-001",
       "https://classes.berkeley.edu/content/2020-spring-geog-200b-001-sem-001"]
    
result = course_info(url_list)
info = result.information
info.head()

,Course_No.,Serial_NO.,Course_Name,Location,Start_Time(SF_time),End_Time(SF_time),Start_Time(TW_time),End_Time(TW_time),Level,Type,Mode,Instructor(s),Units,Total_Capacity,Total_Enrolled,Final_Examination,Description
0,2020 Spring GEOG 149B 001 LEC 001,31059,Climate Impacts and Risk Analysis,McCone 145,Mon 11:00 / Wed 11:00,Mon 12:30 / Wed 12:30,Tue 02:00 / Thu 02:00,Tue 03:30 / Thu 03:30,Undergraduate,Lecture,Pending Reviews,Norman L Miller,3,50,39,Alternative method of final assessment,Climate impacts and risk analysis is the study...
1,2020 Spring GEOG 167AC 001 LEC 001,30924,"Border Geographies, Migration and Decolonial M...",Genetics & Plant Bio 100,Tue 12:30 / Thu 12:30,Tue 13:59 / Thu 13:59,Wed 03:30 / Fri 03:30,Wed 04:59 / Fri 04:59,Undergraduate,Lecture,Pending Reviews,Diana M Negrin,4,200,66,Alternative method of final assessment,This course examines how today’s bounded geogr...
2,2020 Spring GEOG 200B 001 SEM 001,19853,Contemporary Geographic Thought 2 (Geographica...,McCone 509,Wed 09:00,Wed 12:00,Thu 00:00,Thu 03:00,Graduate,Seminar,Pending Reviews,Sharad Chari,5,14,6,No final exam,'Geographical Difference/Differentiation' is a...


In [ ]:
# copy the result and paste it on your excel/Google Sheet/Database
info.to_clipboard(excel=True,sep='\t')

In [ ]:
# or save file as .xlsx directly
multiple_results.to_excel('course_info.xlsx', na_rep=False)

### 2. If you wanna export a schedule of all selected courses

In [4]:
schedule = result.weekly_schedule(time_zone="SF")

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
00:00:00,,,,,,
00:30:00,,,,,,
01:00:00,,,,,,
01:30:00,,,,,,
02:00:00,,,,,,
02:30:00,,,,,,
03:00:00,,,,,,
03:30:00,,,,,,
04:00:00,,,,,,
04:30:00,,,,,,


In [ ]:
# copy the schedule and paste it on your excel/Google Sheet/Database
schedule.to_clipboard(excel=True, sep='\t')

In [ ]:
# or save file as .xlsx directly
schedule.to_excel('course_schedule.xlsx', na_rep=False)

## Search for the courses that interest you

In [ ]:
# NOW, Try it on your own with course of your interest!
url_list = ["",
            "",
            "",
            "",
            ""] # insert links, you can add as more links as you want
result = course_info(url_list)
info = result.information
info.head()
# copy the result and paste it on your excel/Google Sheet/Database
info.to_clipboard(excel=True,sep='\t')
